In [1]:
import requests
from obi_auth import get_token
from entitysdk import Client, ProjectContext, models
from enum import Enum

class DatabaseEnvironment(Enum):
    STAGING_DATABASE = "staging"
    # PRODUCTION_DATABASE = "production"
    # LOCAL_UNAUTHENTICATED = "local_unauthenticated"
    # Note: LOCAL_AUTHENTICATED is not supported by entitycore because the project/vlab ids do not exist in your keycloak groups

obi_one_api_url = "http://127.0.0.1:8100"

add_morphology_asset = False

database_environment = DatabaseEnvironment.STAGING_DATABASE
# database_environment = DatabaseEnvironment.PRODUCTION_DATABASE
# database_environment = DatabaseEnvironment.LOCAL_UNAUTHENTICATED

if database_environment == DatabaseEnvironment.STAGING_DATABASE:
    """
    - Uncomment in obi-one/.env.run-local before "make run-local": 
        export ENTITYCORE_URL=https://staging.openbraininstitute.org/api/entitycore
    """
    entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"
    virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
    project_id="2720f785-a3a2-4472-969d-19a53891c817"

# elif database_environment == DatabaseEnvironment.PRODUCTION_DATABASE:
#     """
#     For future: EntityCore not currently in production.
#     - Uncomment in obi-one/.env.run-local before "make run-local": 
#         export ENTITYCORE_URL=https://www.openbraininstitute.org/api/entitycore
#     """
#     entitycore_api_url = "https://www.openbraininstitute.org/api/entitycore"

elif database_environment == DatabaseEnvironment.LOCAL_UNAUTHENTICATED:
    """
    Not yet tested.
    - Launch entitycore locally (make run-local)
    - Add a morphology with a project name (see entitysdk)
    """
    entitycore_api_url = "https://staging.openbraininstitute.org/api/entitycore"
    add_morphology_asset = True
    virtual_lab_id="a98b7abc-fc46-4700-9e3d-37137812c730"
    project_id="0dbced5f-cc3d-488a-8c7f-cfb8ea039dc6"
    # virtual_lab_id="e6030ed8-a589-4be2-80a6-f975406eb1f6"
    # project_id="2720f785-a3a2-4472-969d-19a53891c817"
    
else:
    raise ValueError(f"Unsupported environment: {database_environment}")


token = get_token(environment="staging")
project_context = ProjectContext(virtual_lab_id=virtual_lab_id, project_id=project_id)
client = Client(api_url=entitycore_api_url, project_context=project_context, token_manager=token)

# Create SimulationForm using Cirucit

In [ ]:
import obi_one as obi

# Sim duration
sim_duration = 3000.0

# Empty Simulation Configuration
sim_conf = obi.SimulationsForm.empty_config()

# Info
info = obi.Info(name="O1 Simulation", description="Simulation of O1 circuit with predefined neuron set and constant current stimulus")
sim_conf.set(info, name="info")

# Neuron Sets
sim_neuron_set = obi.IDNeuronSet(neuron_ids=obi.NamedTuple(name="IDNeuronSet1", elements=range(10)))
sim_conf.add(sim_neuron_set, name='L1All')

# Regular Timesteps
regular_timestamps = obi.RegularTimestamps(start_time=0.0, number_of_repetitions=3, interval=sim_duration)
sim_conf.add(regular_timestamps, name='RegularTimestamps')

# Stimulus
poisson_input = obi.PoissonSpikeStimulus(timestamps=regular_timestamps.ref, stim_duration=800, frequency=20, neuron_set=sim_neuron_set.ref)
sim_conf.add(poisson_input, name='PoissonInputStimulus')

# Recordings
voltage_recording = obi.SomaVoltageRecording(neuron_set=sim_neuron_set.ref, start_time=0.0, end_time=sim_duration)
sim_conf.add(voltage_recording, name='VoltageRecording')

# Initialization
simulations_initialize = obi.SimulationsForm.Initialize(circuit=obi.CircuitFromID(id_str="662dce5c-306d-4737-acd6-a3af985aad7f"), 
                                                        node_set=sim_neuron_set.ref, 
                                                        simulation_length=sim_duration)
sim_conf.set(simulations_initialize, name='initialize')

# Validated Config
validated_sim_conf = sim_conf.validated_config()

# Generate GridScan 

In [ ]:
import tempfile

# Create a temporary directory (not used in this example, but can be useful for testing)
with tempfile.TemporaryDirectory() as temp_dir:

    """
    Very strange issue with adding assets when a relative path is used here. Need to look into after CNS.
    """
    # grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='../../obi-output/circuit_simulations/grid_scan')
    grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root='/Users/james/Documents/obi/code/obi-output/circuit_simulations/grid_scan')
    # grid_scan = obi.GridScan(form=validated_sim_conf, coordinate_directory_option="ZERO_INDEX", output_root=temp_dir)
    grid_scan.multiple_value_parameters(display=True)
    grid_scan.coordinate_parameters(display=True)
    grid_scan.execute(processing_method='generate', data_postprocessing_method='save', db_client=client)

# Test endpoint. Service must be launched first (make run-local)

In [ ]:
import requests

# Construct the full endpoint URL
url = f"{obi_one_api_url}/generated/simulations-generate-grid-save"
# Prepare headers
headers = {
    "Authorization": f"Bearer {token}",
    "Accept": "application/json",
    "Content-Type": "application/json"
}

# Optionally include optional headers if they are set
if virtual_lab_id:
    headers["virtual-lab-id"] = virtual_lab_id
if project_id:
    headers["project-id"] = project_id

# Construct request body — adjust this with the actual morphology metrics form data!
# Placeholder example:
request_body = {
 'type': 'SimulationsForm',
 'timestamps': {'RegularTimestamps': {'type': 'RegularTimestamps',
   'start_time': 0.0,
   'number_of_repetitions': 3,
   'interval': 3000.0}},
 'stimuli': {'PoissonInputStimulus': {'type': 'PoissonSpikeStimulus',
   'timestamps': {'type': 'TimestampsReference',
    'block_dict_name': 'timestamps',
    'block_name': 'RegularTimestamps'},
   'stim_duration': 800.0,
   'neuron_set': {'type': 'NeuronSetReference',
    'block_dict_name': 'neuron_sets',
    'block_name': 'L1All'},
   'random_seed': 0,
   'frequency': 20.0}},
 'recordings': {'VoltageRecording': {'type': 'SomaVoltageRecording',
   'start_time': 0.0,
   'end_time': 3000.0,
   'dt': 0.1,
   'neuron_set': {'type': 'NeuronSetReference',
    'block_dict_name': 'neuron_sets',
    'block_name': 'L1All'}}},
 'neuron_sets': {'L1All': {'type': 'IDNeuronSet',
   'random_sample': None,
   'random_seed': 0,
   'neuron_ids': {'type': 'NamedTuple',
    'name': 'IDNeuronSet1',
    'elements': (0, 1, 2, 3, 4, 5, 6, 7, 8, 9)}}},
 'initialize': {'type': 'SimulationsForm.Initialize',
  'circuit': {'type': 'CircuitFromID',
   'id_str': '662dce5c-306d-4737-acd6-a3af985aad7f'},
  'simulation_length': 3000.0,
  'node_set': {'type': 'NeuronSetReference',
   'block_dict_name': 'neuron_sets',
   'block_name': 'L1All'},
  'random_seed': 1,
  'extracellular_calcium_concentration': 1.1,
  'v_init': -80.0},
 'info': {'type': 'Info',
  'campaign_name': 'No name provided',
  'campaign_description': 'No description provided'}
}

# Make the POST request
response = requests.post(url, headers=headers, json=request_body)

# Check the response
if response.status_code == 200:
    data = response.json()
    simulation_campaign_id = data
    print("Success:", data)
else:
    print(f"Error {response.status_code}: {response.text}")

# Stage the Circuit

In [ ]:
from entitysdk.models import Circuit
circuit = client.get_entity(entity_id="662dce5c-306d-4737-acd6-a3af985aad7f", entity_type=Circuit)

from entitysdk.staging.circuit import stage_circuit
from pathlib import Path
stage_circuit(client=client, model=circuit, output_dir=Path('temp_circ/'))

# Stage the Simulation

In [ ]:
from entitysdk.models import SimulationCampaign
simulation_campaign = client.get_entity(entity_id=simulation_campaign_id, entity_type=SimulationCampaign)
simulation = simulation_campaign.simulations[0]


from entitysdk.staging.simulation import stage_simulation
stage_simulation(client=client, model=simulation, output_dir=Path('temp_sim'), circuit_config_path=Path("temp_circ/circuit_config.json"))

In [6]:
import entitysdk
from obi_one import CircuitFromID
import tempfile
from pathlib import Path

_campaign = client.register_entity(
    entitysdk.models.SimulationCampaign(
        name="Hello",
        description="World",
        entity_id="662dce5c-306d-4737-acd6-a3af985aad7f",
        scan_parameters={},
    )
)

simulation = client.register_entity(
    entitysdk.models.Simulation(
        name="sim-0",
        description="sim-0",
        scan_parameters={"foo": "bar"},
        entity_id="662dce5c-306d-4737-acd6-a3af985aad7f",
        simulation_campaign_id=_campaign.id
    )
)

with tempfile.TemporaryDirectory() as tdir:
    file1 = Path(tdir, "run_coordinate_instance.json")
    file1.write_text("hello")
    _ = client.upload_file(
                entity_id=simulation.id,
                entity_type=entitysdk.models.Simulation,
                file_path=file1,
                file_content_type="application/json",
                asset_label='simulation_generation_config'
            )
    
simulation_campaign = client.get_entity(entity_id=_campaign.id, entity_type=entitysdk.models.SimulationCampaign)
print(simulation_campaign.simulations[0])

with tempfile.TemporaryDirectory() as tdir:
    file1 = Path(tdir, "run_coordinate_instance.json")
    file1.write_text("hello")
    _ = client.upload_file(
                entity_id=simulation.id,
                entity_type=entitysdk.models.Simulation,
                file_path=file1,
                file_content_type="application/json",
                asset_label='simulation_generation_config'
            )

[2025-06-24 10:29:43,996] INFO: HTTP Request: POST https://staging.openbraininstitute.org/api/entitycore/simulation-campaign "HTTP/1.1 200 OK"
[2025-06-24 10:29:44,111] INFO: HTTP Request: POST https://staging.openbraininstitute.org/api/entitycore/simulation "HTTP/1.1 200 OK"
[2025-06-24 10:29:44,324] INFO: HTTP Request: POST https://staging.openbraininstitute.org/api/entitycore/simulation/9003eb96-c646-4fb7-8b1a-e130fd041a67/assets "HTTP/1.1 201 Created"
[2025-06-24 10:29:44,445] INFO: HTTP Request: GET https://staging.openbraininstitute.org/api/entitycore/simulation-campaign/26826136-d195-4f05-a789-c7e71c9387eb "HTTP/1.1 200 OK"
assets=[] id=UUID('9003eb96-c646-4fb7-8b1a-e130fd041a67') update_date=None creation_date=None name='sim-0' description='sim-0' type='simulation' created_by=None updated_by=None authorized_public=False authorized_project_id=None contributions=None legacy_id=None simulation_campaign_id=UUID('26826136-d195-4f05-a789-c7e71c9387eb') entity_id=UUID('662dce5c-306d-4

EntitySDKError: HTTP error 409 for POST https://staging.openbraininstitute.org/api/entitycore/simulation/9003eb96-c646-4fb7-8b1a-e130fd041a67/assets
data       : {'label': 'simulation_generation_config'}
json       : null
params     : None
response   : {"error_code":"ASSET_DUPLICATED","message":"Asset with path 'run_coordinate_instance.json' already exists","details":null}

# Trying to reproduce relative path error (please ignore)

In [ ]:
# import entitysdk

# _campaign = client.register_entity(
#             entitysdk.models.SimulationCampaign(
#                 name="Hello",
#                 description="World",
#                 entity_id="662dce5c-306d-4737-acd6-a3af985aad7f",
#                 scan_parameters={},
#             )
#         )

# from pathlib import Path

# model_dump = {"foo": "bar", "baz": 42}
# import json



# with tempfile.TemporaryDirectory() as tdir:

#     output_path = Path("../run_scan_config.json")
#     # output_path = Path("/Users/james/Documents/obi/code/obi-one/obi-output/run_scan_config.json")
#     # output_path = Path(tdir, "run_scan_config.json")

#     output_path.write_text("swc")

#     _ = client.upload_file(
#         entity_id=_campaign.id,
#         entity_type=entitysdk.models.SimulationCampaign,
#         file_path=output_path,
#         file_content_type="application/json",
#         asset_label='campaign_generation_config'
#     )